In [1]:
import pandas as pd
import numpy as np
import torch
test_df=pd.read_csv('kaggle_data/test.csv')
test_df=test_df.astype(np.float32)
test_np_X=test_df.values
test_np_X=test_np_X.reshape([28000,1,28,28])
test_np_X=(test_np_X/255-0.5)/0.5
test_X=torch.from_numpy(test_np_X)

In [2]:
import torch.nn as nn
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1=nn.Conv2d(1,6,5,padding=2)
        self.conv2=nn.Conv2d(6,16,5)
        self.pool=nn.MaxPool2d(2,2)
        self.fc1=nn.Linear(400,120)
        self.fc2=nn.Linear(120,84)
        self.out=nn.Linear(84,10)
    def forward(self,x):
        x=self.pool(torch.tanh(self.conv1(x)))
        x=self.pool(torch.tanh(self.conv2(x)))
        x=x.reshape(-1,self.num_flat_features(x))
        x=torch.tanh(self.fc1(x))
        x=torch.tanh(self.fc2(x))
        x=torch.softmax(self.out(x),dim=-1)
        return x
    def num_flat_features(self,x):
        size=x.size()[1:]
        num_features=1
        for s in size:
            num_features*=s
        return num_features
cnn=CNN()
cnn

CNN(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (out): Linear(in_features=84, out_features=10, bias=True)
)

In [3]:
device=torch.device('cuda:0')
cnn=cnn.to(device)

In [4]:
state_dict=torch.load('saves/kaggle_cnn_params.pth')
cnn.load_state_dict(state_dict)

<All keys matched successfully>

In [5]:
cnn(test_X[0].reshape(1,1,28,28).to(device))[0]

tensor([7.5855e-25, 1.0665e-29, 1.0000e+00, 1.3208e-15, 1.9727e-25, 6.0855e-33,
        7.7360e-19, 2.1410e-25, 5.1865e-22, 2.3471e-30], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [6]:
Y_pred=cnn(test_X.to(device))
ans=[]
for i in range(len(Y_pred)):
    max_pos=0
    max_num=0
    for j in range(10):
        if max_num<Y_pred[i][j]:
            max_num=Y_pred[i][j]
            max_pos=j
    ans.append(max_pos)

In [7]:
Y_submit=pd.DataFrame({'ImageId':range(1,len(ans)+1),'Label':ans})
Y_submit

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [8]:
Y_submit.to_csv('submit/submission.csv',index=False)

最终在 kaggle 上跑了 0.98371
比在训练集上还高...
说起来神经网络是不是不会像ML那样那么容易过拟合呢
我这都没做验证集
总之这个结果我觉得挺不错的，用ML我应该很难做到那么好，所以事实证明DL还是无脑强
然后就是，我对吴恩达课程的理解思路可以说基本正确，对lenet-5的优化方向没有搞错
这样以后最核心的事情实际上我觉得应该是多看论文，积累经验吧
说到底网络架构就是多看别人的好与坏，理解理解，抄抄改改